### AEP

In [2]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import prophet_linear_adjust as prophet_based

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [ ]:
def simple_dst_fix(df: pd.DataFrame, start_at_midnight: bool = True) -> pd.DataFrame:
    df = df.copy()
    df["ds"] = pd.to_datetime(df["ds"])
    df = df.sort_values("ds")

    df = df[~df["ds"].duplicated(keep="first")]

    start = df["ds"].iloc[0]
    if start_at_midnight:
        start = start.normalize()
    end = df["ds"].iloc[-1]
    full_idx = pd.date_range(start, end, freq="h")

    out = df.set_index("ds").reindex(full_idx)

    num_cols = out.select_dtypes(include="number").columns
    out[num_cols] = out[num_cols].ffill()

    if out[num_cols].isna().any().any():
        out[num_cols] = out[num_cols].bfill()

    out = out.rename_axis("ds").reset_index()

    return out

In [4]:
df_aep = pd.read_csv("data/AEP_hourly.csv")
df_aep.rename(columns={'Datetime': 'ds', 'AEP_MW': 'y'}, inplace=True)
df_aep['ds'] = pd.to_datetime(df_aep['ds'], format='%Y-%m-%d %H:%M:%S')
df_aep = simple_dst_fix(df_aep)
out_dir = 'AEP_results'

### test propht model

In [5]:
import prophet_linear_adjust_yearly_to_hourly

yearly_demand = df_aep.groupby(df_aep['ds'].dt.year)['y'].sum().reset_index()
date_start = pd.to_datetime('2005-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

results_prophet_yearly = []
i = 1

while True:
    print(i)
    result = prophet_linear_adjust_yearly_to_hourly.forecast_next_year_hourly(
        df_aep, date_start, date_end, yearly_demand,
        bayesian_samples=0,
        manual=False,
        daily=True,
        weekly = True,
        yearly = True,
        monthly = True)

    results_prophet_yearly.append(result)

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    if i == 3:
        break
    else:
        i += 1

results_prophet_yearly = pd.concat(results_prophet_yearly, ignore_index=True)
results_prophet_yearly.to_csv('experiment_results/' + out_dir + '/results_prophet_yearly.csv', index=False)


1


02:52:18 - cmdstanpy - INFO - Chain [1] start processing
02:52:37 - cmdstanpy - INFO - Chain [1] done processing


2


02:57:48 - cmdstanpy - INFO - Chain [1] start processing
02:58:11 - cmdstanpy - INFO - Chain [1] done processing


3


03:02:56 - cmdstanpy - INFO - Chain [1] start processing
Python(48722) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
03:03:16 - cmdstanpy - INFO - Chain [1] done processing


### test RNN augmented Prophet model

In [6]:
import pandas as pd
import numpy as np
import torch
import prophet_yearly_to_daily
import RNN_fourier_RNN_uncertainty

train_config = RNN_fourier_RNN_uncertainty.training_config(n_epochs=30, device=torch.device("mps"))
fourier_conf = RNN_fourier_RNN_uncertainty.fourier_config(mode="matrix", K_weekly=3, K_monthly=6, K_yearly=10)

K_total = fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly
F_per_hour = 2 * K_total

if fourier_conf.mode == "vector":
    cont_dim = 1 + F_per_hour
    fourier_dim = F_per_hour
else:
    cont_dim = 1 + 24 * F_per_hour
    fourier_dim = F_per_hour

df_daily = df_aep.copy()
df_daily['ds'] = pd.to_datetime(df_daily['ds'])
df_daily['date'] = df_daily['ds'].dt.date
df_daily_agg = df_daily.groupby('date')['y'].sum().reset_index()
df_daily_agg.columns = ['ds', 'y']
df_daily_agg['ds'] = pd.to_datetime(df_daily_agg['ds'])

yearly_demand = df_daily_agg.copy()
yearly_demand['year'] = yearly_demand['ds'].dt.year
yearly_demand = yearly_demand.groupby('year')['y'].sum().reset_index()
yearly_demand.columns = ['ds', 'y']

results_hybrid = pd.DataFrame({'ds': [], 'y_hat': [], 'y': []})

date_start = pd.to_datetime('2005-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

i = 1

while True:

    print(i)
    df_train_day = df_daily_agg.loc[(df_daily_agg['ds'] >= date_start) & (df_daily_agg['ds'] < date_end)].copy()
    df_test_day = df_daily_agg.loc[(df_daily_agg['ds'] >= date_end) & (df_daily_agg['ds'] < date_end + pd.DateOffset(years=1))].copy()

    # ===== STAGE 1: Prophet predicts daily loads for the next year =====
    result_prophet = prophet_yearly_to_daily.forecast_next_year_daily(
        df_train_day,
        df_test_day,
        yearly_demand,
        manual=False,
        weekly=True,
        monthly=True,
        yearly=True
    )
    result_prophet = result_prophet.sort_values('ds').reset_index()

    df_train_hour = df_aep.loc[(df_aep['ds'] >= date_start) & (df_aep['ds'] < date_end)].copy().sort_values('ds').reset_index(drop=True)
    df_test_hour = df_aep.loc[(df_aep['ds'] >= date_end) & (df_aep['ds'] < date_end + pd.DateOffset(years=1))].copy().sort_values('ds').reset_index(drop=True)

    model = RNN_fourier_RNN_uncertainty.RNN_fourier(
        cont_dim=cont_dim,
        fourier_dim=fourier_dim,
        xf_mode="matrix",
        d_model = 128,
        latent_dim=32,
        nhead=4
    )

    trainer = RNN_fourier_RNN_uncertainty.RNN_train_fourier(model, train_config, fourier_conf)
    trainer(df_train_hour)

    daily_prediction = result_prophet[['ds','adjusted_y']]
    daily_prediction.columns = ['ds', 'y']

    fake_hourly_load = []
    for _, row in daily_prediction.iterrows():
        day = row['ds']
        daily_val = row['y'] / 24
        # create 24 hourly entries for this day
        for h in range(24):
            fake_hourly_load.append({
                'ds': pd.Timestamp(day) + pd.Timedelta(hours=h),
                'y': daily_val
            })

    # convert to DataFrame
    fake_hourly_load = pd.DataFrame(fake_hourly_load)

    forcast, _ = trainer.forecate(fake_hourly_load, deterministic = True)

    result = pd.DataFrame({'ds': df_test_hour['ds'], 'y_hat': forcast, 'y': df_test_hour['y']})

    results_hybrid = pd.concat([results_hybrid, result]).reset_index(drop=True)

    results_hybrid.to_csv('experiment_results/' + out_dir + '/results_prophet_rnn_hybrid.csv', index=False)

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    if i  == 3:
        break

    else:
        i = i + 1

results_hybrid.to_csv('experiment_results/' + out_dir + '/results_prophet_rnn_hybrid.csv', index=False)

04:28:34 - cmdstanpy - INFO - Chain [1] start processing


1


04:28:34 - cmdstanpy - INFO - Chain [1] done processing


epoch 1 loss: 0.1635
epoch 2 loss: 0.0772
epoch 3 loss: 0.0571
epoch 4 loss: 0.0459
epoch 5 loss: 0.0440
epoch 6 loss: 0.0522
epoch 7 loss: 0.0415
epoch 8 loss: 0.0373
epoch 9 loss: 0.0342
epoch 10 loss: 0.0408
epoch 11 loss: 0.0394
epoch 12 loss: 0.0374
epoch 13 loss: 0.0379
epoch 14 loss: 0.0370
epoch 15 loss: 0.0352
epoch 16 loss: 0.0380
epoch 17 loss: 0.0317
epoch 18 loss: 0.0353
epoch 19 loss: 0.0356
epoch 20 loss: 0.0366
epoch 21 loss: 0.0402
epoch 22 loss: 0.0348
epoch 23 loss: 0.0403
epoch 24 loss: 0.0330
epoch 25 loss: 0.0376
epoch 26 loss: 0.0367
epoch 27 loss: 0.0346
epoch 28 loss: 0.0301
epoch 29 loss: 0.0361
epoch 30 loss: 0.0335


/var/folders/3h/5yfj7yqx7lg98t310bfxxr3m0000gn/T/ipykernel_50767/691296404.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_hybrid = pd.concat([results_hybrid, result]).reset_index(drop=True)
04:33:10 - cmdstanpy - INFO - Chain [1] start processing


...

epoch 1 loss: 0.1827
epoch 2 loss: 0.0743
epoch 3 loss: 0.0538
epoch 4 loss: 0.0458
epoch 5 loss: 0.0448
epoch 6 loss: 0.0371
epoch 7 loss: 0.0407
epoch 8 loss: 0.0376
epoch 9 loss: 0.0365
epoch 10 loss: 0.0384
epoch 11 loss: 0.0386
epoch 12 loss: 0.0368
epoch 13 loss: 0.0381
epoch 14 loss: 0.0370
epoch 15 loss: 0.0392
epoch 16 loss: 0.0398
epoch 17 loss: 0.0333
epoch 18 loss: 0.0359
epoch 19 loss: 0.0325
epoch 20 loss: 0.0325
epoch 21 loss: 0.0347
epoch 22 loss: 0.0309
epoch 23 loss: 0.0353
epoch 24 loss: 0.0300
epoch 25 loss: 0.0342
epoch 26 loss: 0.0307
epoch 27 loss: 0.0305
epoch 28 loss: 0.0325
epoch 29 loss: 0.0299
epoch 30 loss: 0.0307


04:38:41 - cmdstanpy - INFO - Chain [1] start processing


3


04:38:41 - cmdstanpy - INFO - Chain [1] done processing


epoch 1 loss: 0.0875
epoch 2 loss: 0.0515
epoch 3 loss: 0.0425
epoch 4 loss: 0.0414
epoch 5 loss: 0.0371
epoch 6 loss: 0.0384
epoch 7 loss: 0.0341
epoch 8 loss: 0.0385
epoch 9 loss: 0.0368
epoch 10 loss: 0.0385
epoch 11 loss: 0.0384
epoch 12 loss: 0.0397
epoch 13 loss: 0.0297
epoch 14 loss: 0.0406
epoch 15 loss: 0.0348
epoch 16 loss: 0.0334
epoch 17 loss: 0.0337
epoch 18 loss: 0.0302
epoch 19 loss: 0.0336
epoch 20 loss: 0.0306
epoch 21 loss: 0.0340
epoch 22 loss: 0.0349
epoch 23 loss: 0.0328
epoch 24 loss: 0.0328
epoch 25 loss: 0.0334
epoch 26 loss: 0.0316
epoch 27 loss: 0.0316
epoch 28 loss: 0.0296
epoch 29 loss: 0.0337
epoch 30 loss: 0.0323


### test 2 stage RNN

In [29]:
import pandas as pd
import numpy as np
import torch
import Hierarchical_RNN_2stage_uncertainty as hrnn
import os


# ============================================================================
# Configuration
# ============================================================================

fourier_config = hrnn.FourierConfig(
    K_yearly_to_daily=8,
    K_monthly_to_daily=6,
    K_weekly_to_daily=8,
    K_yearly_to_hourly=10,
    K_monthly_to_hourly=7,
    K_daily_to_hourly=8,
    K_weekly_to_hourly=3,
)

train_config = hrnn.TrainingConfig(
    base_epochs=200,
    yearly_to_daily_multiplier=6,
    daily_to_hourly_multiplier=1,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    lr=5e-4,
    lambda0=1e-6,
    lambdaf=1e-5,
    batch_size=32,
    T_seq_yearly=3,
    T_seq_daily=32,
    curriculum_start_prob=1.0,
    curriculum_end_prob=1.0
)

# ============================================================================
# Results Storage
# ============================================================================

results_hierarchical = pd.DataFrame({
    'year': [],
    'month': [],
    'day': [],
    'hour': [],
    'y_hat': [],
    'y': []
})

# ============================================================================
# Rolling Window Setup
# ============================================================================

date_start = pd.to_datetime('2008-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

print("=" * 70)
print("2-Stage Hierarchical RNN - Deterministic Prediction")
print("=" * 70)
print(f"Initial training window: {date_start.date()} to {date_end.date()}")
print(f"Window size: ~10 years")
print(f"Prediction: 1 year ahead (deterministic)")
print(f"T_seq_yearly: {train_config.T_seq_yearly}")
print(f"T_seq_daily: {train_config.T_seq_daily}")
print(f"Device: {train_config.device}")
print("=" * 70)

# Create output directory
os.makedirs(f'experiment_results/{out_dir}', exist_ok=True)

# ============================================================================
# Rolling Window Testing Loop
# ============================================================================

i = 1

while True:
    print("\n" + "=" * 70)
    print(f"Iteration {i}")
    print("=" * 70)

    # Get training data
    df_train = df_aep.loc[(df_aep['ds'] >= date_start) & (df_aep['ds'] < date_end)].copy().sort_values(by='ds').reset_index(drop=True)

    # Get test year
    test_year_start = date_end
    test_year_end = date_end + pd.DateOffset(years=1)
    df_test = df_aep.loc[(df_aep['ds'] >= test_year_start) & (df_aep['ds'] < test_year_end)].copy().sort_values(by='ds').reset_index(drop=True)

    if len(df_test) == 0:
        print("No more test data available. Stopping.")
        break

    print(f"Training: {date_start.date()} to {date_end.date()}")
    print(f"Testing:  {test_year_start.date()} to {test_year_end.date()}")
    print(f"Training samples: {len(df_train)}")
    print(f"Test samples: {len(df_test)}")

    # ========================================================================
    # Build and Train Model
    # ========================================================================

    print("\nBuilding model...")
    model_config = dict(
        fourier_config=fourier_config,
        training_config=train_config,
        latent_dim=36,
        d_model=128,
        nhead=4
    )
    downscaler = hrnn.build_model_dp(hrnn.HierarchicalDownscaler_2Stage, **model_config)

    # Train model using Approach 2 (Curriculum Learning)
    print("\nTraining model...")
    trainer = hrnn.Approach2_CurriculumLearning_2Stage(downscaler)
    trainer.train(df_train)

    # ========================================================================
    # Generate Deterministic Prediction
    # ========================================================================

    test_year_int = test_year_start.year
    yearly_sum = df_test['y'].sum()

    print(f"\nTest year: {test_year_int}")
    print(f"Actual yearly sum: {yearly_sum:.2f}")

    # Prepare historical data from training
    data_dict_train = downscaler.module.prepare_data(df_train) if hasattr(downscaler, 'module') else downscaler.prepare_data(df_train)

    # Get yearly history for Stage 1
    df_yearly_train = data_dict_train['yearly'].sort_values('year')
    historical_years = [{'year': row['year'], 'yearly_sum': row['yearly_sum']} for _, row in df_yearly_train.iterrows()]

    print(f"Using {len(historical_years)} historical years as context")

    print(f"\nGenerating deterministic prediction for year {test_year_int}...")
    predictor = hrnn.HierarchicalPredictor_2Stage(downscaler)

    y_hat = predictor.predict_yearly_to_hourly(
        yearly_sum=yearly_sum,
        year=test_year_int,
        historical_years=historical_years
    )

    print(f"Generated predictions of shape {y_hat.shape}")

    # ========================================================================
    # Align with Test Data
    # ========================================================================

    n_hours_actual = len(df_test)
    n_hours_pred = len(y_hat)

    if n_hours_pred > n_hours_actual:
        y_hat = y_hat[:n_hours_actual]
        print(f"Trimmed predictions from {n_hours_pred} to {n_hours_actual} hours")
    elif n_hours_pred < n_hours_actual:
        # Pad if needed
        padding_value = y_hat.mean()
        pad_width = n_hours_actual - n_hours_pred
        y_hat = np.pad(y_hat, (0, pad_width), 'constant', constant_values=padding_value)
        print(f"Padded predictions from {n_hours_pred} to {n_hours_actual} hours")

    # ========================================================================
    # Store Results
    # ========================================================================

    y_true = df_test['y'].values

    result = pd.DataFrame({
        'year': df_test['ds'].dt.year,
        'month': df_test['ds'].dt.month,
        'day': df_test['ds'].dt.day,
        'hour': df_test['ds'].dt.hour,
        'y_hat': y_hat,
        'y': y_true
    })

    # Append to results
    results_hierarchical = pd.concat(
        [results_hierarchical, result],
        ignore_index=True
    )

    # ========================================================================
    # Save Intermediate Results
    # ========================================================================

    if i % 1 == 0:
        print(f"\nSaving intermediate results...")
        results_hierarchical.to_csv(
            f'experiment_results/{out_dir}/results_hierarchical.csv',
            index=False
        )

    # ========================================================================
    # Move Window Forward
    # ========================================================================

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    # Stop after 3 years of testing
    if i == 3:
        print('\n' + '=' * 70)
        print('Finished 3 iterations!')
        print('=' * 70)
        break
    else:
        i += 1

# ============================================================================
# Final Save
# ============================================================================

print("\n" + "=" * 70)
print("Saving final results...")
print("=" * 70)

results_hierarchical.to_csv(
    f'experiment_results/{out_dir}/results_hierarchical.csv',
    index=False
)

print("\nAll results saved successfully!")
print(f"Output directory: experiment_results/{out_dir}/")
print("\nFiles created:")
print("  - results_hierarchical.csv")

8760


### Comed

In [2]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [3]:
def simple_dst_fix(df: pd.DataFrame, start_at_midnight: bool = True) -> pd.DataFrame:
    df = df.copy()
    df["ds"] = pd.to_datetime(df["ds"])
    df = df.sort_values("ds")

    df = df[~df["ds"].duplicated(keep="first")]

    start = df["ds"].iloc[0]
    if start_at_midnight:
        start = start.normalize()
    end = df["ds"].iloc[-1]
    full_idx = pd.date_range(start, end, freq="h")

    out = df.set_index("ds").reindex(full_idx)

    num_cols = out.select_dtypes(include="number").columns
    out[num_cols] = out[num_cols].ffill()

    if out[num_cols].isna().any().any():
        out[num_cols] = out[num_cols].bfill()

    out = out.rename_axis("ds").reset_index()

    return out

In [4]:
df_comed = pd.read_csv("data/COMED_hourly.csv")
df_comed.rename(columns={'Datetime': 'ds', 'COMED_MW': 'y'}, inplace=True)
df_comed['ds'] = pd.to_datetime(df_comed['ds'], format='%Y-%m-%d %H:%M:%S')
df_comed = simple_dst_fix(df_comed)
out_dir = 'Comed_results'

### test prophet

In [5]:
import prophet_linear_adjust_yearly_to_hourly

yearly_demand = df_comed.groupby(df_comed['ds'].dt.year)['y'].sum().reset_index()
date_start = pd.to_datetime('2011-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

results_prophet_yearly = []
i = 1

while True:
    print(i)
    result = prophet_linear_adjust_yearly_to_hourly.forecast_next_year_hourly(
        df_comed, date_start, date_end, yearly_demand,
        bayesian_samples=0,
        manual=False,
        daily=True,
        weekly = True,
        yearly = True,
        monthly = True)

    results_prophet_yearly.append(result)

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    if i == 3:
        break
    else:
        i += 1

results_prophet_yearly = pd.concat(results_prophet_yearly, ignore_index=True)
results_prophet_yearly.to_csv('experiment_results/' + out_dir + '/results_prophet_yearly.csv', index=False)


1


04:50:04 - cmdstanpy - INFO - Chain [1] start processing
04:50:17 - cmdstanpy - INFO - Chain [1] done processing


2


04:57:47 - cmdstanpy - INFO - Chain [1] start processing
04:57:59 - cmdstanpy - INFO - Chain [1] done processing


3


05:03:40 - cmdstanpy - INFO - Chain [1] start processing
05:03:49 - cmdstanpy - INFO - Chain [1] done processing


### test RNN augmented Prophet model

In [5]:
import pandas as pd
import numpy as np
import torch
import prophet_yearly_to_daily
import RNN_fourier_RNN_uncertainty

train_config = RNN_fourier_RNN_uncertainty.training_config(n_epochs=30, device=torch.device("mps"))
fourier_conf = RNN_fourier_RNN_uncertainty.fourier_config(mode="matrix", K_weekly=3, K_monthly=6, K_yearly=10)

K_total = fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly
F_per_hour = 2 * K_total

if fourier_conf.mode == "vector":
    cont_dim = 1 + F_per_hour
    fourier_dim = F_per_hour
else:
    cont_dim = 1 + 24 * F_per_hour
    fourier_dim = F_per_hour

df_daily = df_comed.copy()
df_daily['ds'] = pd.to_datetime(df_daily['ds'])
df_daily['date'] = df_daily['ds'].dt.date
df_daily_agg = df_daily.groupby('date')['y'].sum().reset_index()
df_daily_agg.columns = ['ds', 'y']
df_daily_agg['ds'] = pd.to_datetime(df_daily_agg['ds'])

yearly_demand = df_daily_agg.copy()
yearly_demand['year'] = yearly_demand['ds'].dt.year
yearly_demand = yearly_demand.groupby('year')['y'].sum().reset_index()
yearly_demand.columns = ['ds', 'y']

results_hybrid = pd.DataFrame({'ds': [], 'y_hat': [], 'y': []})

date_start = pd.to_datetime('2011-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

i = 1

while True:

    print(i)
    df_train_day = df_daily_agg.loc[(df_daily_agg['ds'] >= date_start) & (df_daily_agg['ds'] < date_end)].copy()
    df_test_day = df_daily_agg.loc[(df_daily_agg['ds'] >= date_end) & (df_daily_agg['ds'] < date_end + pd.DateOffset(years=1))].copy()

    # ===== STAGE 1: Prophet predicts daily loads for the next year =====
    result_prophet = prophet_yearly_to_daily.forecast_next_year_daily(
        df_train_day,
        df_test_day,
        yearly_demand,
        manual=False,
        weekly=True,
        monthly=True,
        yearly=True
    )
    result_prophet = result_prophet.sort_values('ds').reset_index()

    df_train_hour = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy().sort_values('ds').reset_index(drop=True)
    df_test_hour = df_comed.loc[(df_comed['ds'] >= date_end) & (df_comed['ds'] < date_end + pd.DateOffset(years=1))].copy().sort_values('ds').reset_index(drop=True)

    model = RNN_fourier_RNN_uncertainty.RNN_fourier(
        cont_dim=cont_dim,
        fourier_dim=fourier_dim,
        xf_mode="matrix",
        d_model = 128,
        latent_dim=32,
        nhead=4
    )

    trainer = RNN_fourier_RNN_uncertainty.RNN_train_fourier(model, train_config, fourier_conf)
    trainer(df_train_hour)

    daily_prediction = result_prophet[['ds','adjusted_y']]
    daily_prediction.columns = ['ds', 'y']

    fake_hourly_load = []
    for _, row in daily_prediction.iterrows():
        day = row['ds']
        daily_val = row['y'] / 24
        for h in range(24):
            fake_hourly_load.append({
                'ds': pd.Timestamp(day) + pd.Timedelta(hours=h),
                'y': daily_val
            })

    # convert to DataFrame
    fake_hourly_load = pd.DataFrame(fake_hourly_load)

    forcast, _ = trainer.forecate(fake_hourly_load, deterministic = True)

    result = pd.DataFrame({'ds': df_test_hour['ds'], 'y_hat': forcast, 'y': df_test_hour['y']})

    results_hybrid = pd.concat([results_hybrid, result]).reset_index(drop=True)

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    if i  == 3:
        break

    else:
        i = i + 1

results_hybrid.to_csv('experiment_results/' + out_dir + '/results_prophet_rnn_hybrid.csv', index=False)

05:36:57 - cmdstanpy - INFO - Chain [1] start processing
05:36:57 - cmdstanpy - INFO - Chain [1] done processing


1


/Users/zero/Documents/GitHub/RNN_based_downscaling/RNN_fourier_RNN_uncertainty.py:495: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:837.)
  print(f"epoch {ep + 1} loss: {float(loss):.4f}")


epoch 1 loss: 0.4941
epoch 2 loss: 0.1456
epoch 3 loss: 0.1051
epoch 4 loss: 0.0648
epoch 5 loss: 0.0699
epoch 6 loss: 0.0694
epoch 7 loss: 0.0422
epoch 8 loss: 0.0384
epoch 9 loss: 0.0421
epoch 10 loss: 0.0435
epoch 11 loss: 0.0268
epoch 12 loss: 0.0322
epoch 13 loss: 0.0402
epoch 14 loss: 0.0314
epoch 15 loss: 0.0372
epoch 16 loss: 0.0402
epoch 17 loss: 0.0272
epoch 18 loss: 0.0277
epoch 19 loss: 0.0301
epoch 20 loss: 0.0246
epoch 21 loss: 0.0279
epoch 22 loss: 0.0338
epoch 23 loss: 0.0359
epoch 24 loss: 0.0255
epoch 25 loss: 0.0258
epoch 26 loss: 0.0256
epoch 27 loss: 0.0271
epoch 28 loss: 0.0264
epoch 29 loss: 0.0262
epoch 30 loss: 0.0351


/var/folders/3h/5yfj7yqx7lg98t310bfxxr3m0000gn/T/ipykernel_53280/403997022.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_hybrid = pd.concat([results_hybrid, result]).reset_index(drop=True)
05:38:47 - cmdstanpy - INFO - Chain [1] start processing


2


05:38:48 - cmdstanpy - INFO - Chain [1] done processing


epoch 1 loss: 0.3052
epoch 2 loss: 0.2069
epoch 3 loss: 0.1433
epoch 4 loss: 0.1355
epoch 5 loss: 0.1252
epoch 6 loss: 0.0787
epoch 7 loss: 0.0654
epoch 8 loss: 0.1013
epoch 9 loss: 0.0487
epoch 10 loss: 0.0655
epoch 11 loss: 0.0513
epoch 12 loss: 0.0512
epoch 13 loss: 0.0763
epoch 14 loss: 0.0469
epoch 15 loss: 0.0441
epoch 16 loss: 0.0516
epoch 17 loss: 0.0371
epoch 18 loss: 0.0331
epoch 19 loss: 0.0502
epoch 20 loss: 0.0383
epoch 21 loss: 0.0370
epoch 22 loss: 0.0388
epoch 23 loss: 0.0315
epoch 24 loss: 0.0339
epoch 25 loss: 0.0403
epoch 26 loss: 0.0351
epoch 27 loss: 0.0295
epoch 28 loss: 0.0279
epoch 29 loss: 0.0318
epoch 30 loss: 0.0313


05:40:33 - cmdstanpy - INFO - Chain [1] start processing
05:40:33 - cmdstanpy - INFO - Chain [1] done processing


3
epoch 1 loss: 0.4401
epoch 2 loss: 0.2247
epoch 3 loss: 0.1539
epoch 4 loss: 0.1120
epoch 5 loss: 0.0855
epoch 6 loss: 0.0741
epoch 7 loss: 0.0603
epoch 8 loss: 0.0539
epoch 9 loss: 0.0489
epoch 10 loss: 0.0479
epoch 11 loss: 0.0403
epoch 12 loss: 0.0328
epoch 13 loss: 0.0379
epoch 14 loss: 0.0343
epoch 15 loss: 0.0302
epoch 16 loss: 0.0296
epoch 17 loss: 0.0343
epoch 18 loss: 0.0339
epoch 19 loss: 0.0327
epoch 20 loss: 0.0393
epoch 21 loss: 0.0310
epoch 22 loss: 0.0357
epoch 23 loss: 0.0336
epoch 24 loss: 0.0319
epoch 25 loss: 0.0289
epoch 26 loss: 0.0289
epoch 27 loss: 0.0314
epoch 28 loss: 0.0285
epoch 29 loss: 0.0319
epoch 30 loss: 0.0314


### test 2 stage RNN

In [ ]:
import pandas as pd
import numpy as np
import torch
import Hierarchical_RNN_2stage_uncertainty as hrnn
import os


# ============================================================================
# Configuration
# ============================================================================

fourier_config = hrnn.FourierConfig(
    K_yearly_to_daily=8,
    K_monthly_to_daily=6,
    K_weekly_to_daily=8,
    K_yearly_to_hourly=10,
    K_monthly_to_hourly=7,
    K_daily_to_hourly=8,
    K_weekly_to_hourly=3,
)

train_config = hrnn.TrainingConfig(
    base_epochs=200,
    yearly_to_daily_multiplier=6,
    daily_to_hourly_multiplier=1,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    lr=5e-4,
    lambda0=1e-6,
    lambdaf=1e-5,
    batch_size=32,
    T_seq_yearly=3,
    T_seq_daily=32,
    curriculum_start_prob=1.0,
    curriculum_end_prob=1.0
)

# ============================================================================
# Results Storage
# ============================================================================

results_hierarchical = pd.DataFrame({
    'year': [],
    'month': [],
    'day': [],
    'hour': [],
    'y_hat': [],
    'y': []
})

# ============================================================================
# Rolling Window Setup
# ============================================================================

date_start = pd.to_datetime('2011-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

print("=" * 70)
print("2-Stage Hierarchical RNN - Deterministic Prediction")
print("=" * 70)
print(f"Initial training window: {date_start.date()} to {date_end.date()}")
print(f"Window size: ~10 years")
print(f"Prediction: 1 year ahead (deterministic)")
print(f"T_seq_yearly: {train_config.T_seq_yearly}")
print(f"T_seq_daily: {train_config.T_seq_daily}")
print(f"Device: {train_config.device}")
print("=" * 70)

# Create output directory
os.makedirs(f'experiment_results/{out_dir}', exist_ok=True)

# ============================================================================
# Rolling Window Testing Loop
# ============================================================================

i = 1

while True:
    print("\n" + "=" * 70)
    print(f"Iteration {i}")
    print("=" * 70)

    # Get training data
    df_train = df_comed.loc[(df_comed['ds'] >= date_start) & (df_comed['ds'] < date_end)].copy().sort_values(by='ds').reset_index(drop=True)

    # Get test year
    test_year_start = date_end
    test_year_end = date_end + pd.DateOffset(years=1)
    df_test = df_comed.loc[(df_comed['ds'] >= test_year_start) & (df_comed['ds'] < test_year_end)].copy().sort_values(by='ds').reset_index(drop=True)

    if len(df_test) == 0:
        print("No more test data available. Stopping.")
        break

    print(f"Training: {date_start.date()} to {date_end.date()}")
    print(f"Testing:  {test_year_start.date()} to {test_year_end.date()}")
    print(f"Training samples: {len(df_train)}")
    print(f"Test samples: {len(df_test)}")

    # ========================================================================
    # Build and Train Model
    # ========================================================================

    print("\nBuilding model...")
    model_config = dict(
        fourier_config=fourier_config,
        training_config=train_config,
        latent_dim=36,
        d_model=128,
        nhead=4
    )
    downscaler = hrnn.build_model_dp(hrnn.HierarchicalDownscaler_2Stage, **model_config)

    # Train model using Approach 2 (Curriculum Learning)
    print("\nTraining model...")
    trainer = hrnn.Approach2_CurriculumLearning_2Stage(downscaler)
    trainer.train(df_train)

    # ========================================================================
    # Generate Deterministic Prediction
    # ========================================================================

    test_year_int = test_year_start.year
    yearly_sum = df_test['y'].sum()

    print(f"\nTest year: {test_year_int}")
    print(f"Actual yearly sum: {yearly_sum:.2f}")

    # Prepare historical data from training
    data_dict_train = downscaler.module.prepare_data(df_train) if hasattr(downscaler, 'module') else downscaler.prepare_data(df_train)

    # Get yearly history for Stage 1
    df_yearly_train = data_dict_train['yearly'].sort_values('year')
    historical_years = [{'year': row['year'], 'yearly_sum': row['yearly_sum']} for _, row in df_yearly_train.iterrows()]

    print(f"Using {len(historical_years)} historical years as context")

    print(f"\nGenerating deterministic prediction for year {test_year_int}...")
    predictor = hrnn.HierarchicalPredictor_2Stage(downscaler)

    y_hat = predictor.predict_yearly_to_hourly(
        yearly_sum=yearly_sum,
        year=test_year_int,
        historical_years=historical_years
    )

    print(f"Generated predictions of shape {y_hat.shape}")

    # ========================================================================
    # Align with Test Data
    # ========================================================================

    n_hours_actual = len(df_test)
    n_hours_pred = len(y_hat)

    if n_hours_pred > n_hours_actual:
        y_hat = y_hat[:n_hours_actual]
        print(f"Trimmed predictions from {n_hours_pred} to {n_hours_actual} hours")
    elif n_hours_pred < n_hours_actual:
        # Pad if needed
        padding_value = y_hat.mean()
        pad_width = n_hours_actual - n_hours_pred
        y_hat = np.pad(y_hat, (0, pad_width), 'constant', constant_values=padding_value)
        print(f"Padded predictions from {n_hours_pred} to {n_hours_actual} hours")

    # ========================================================================
    # Store Results
    # ========================================================================

    y_true = df_test['y'].values

    result = pd.DataFrame({
        'year': df_test['ds'].dt.year,
        'month': df_test['ds'].dt.month,
        'day': df_test['ds'].dt.day,
        'hour': df_test['ds'].dt.hour,
        'y_hat': y_hat,
        'y': y_true
    })

    # Append to results
    results_hierarchical = pd.concat(
        [results_hierarchical, result],
        ignore_index=True
    )

    # ========================================================================
    # Save Intermediate Results
    # ========================================================================

    if i % 1 == 0:
        print(f"\nSaving intermediate results...")
        results_hierarchical.to_csv(
            f'experiment_results/{out_dir}/results_hierarchical.csv',
            index=False
        )

    # ========================================================================
    # Move Window Forward
    # ========================================================================

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    # Stop after 3 years of testing
    if i == 3:
        print('\n' + '=' * 70)
        print('Finished 3 iterations!')
        print('=' * 70)
        break
    else:
        i += 1

# ============================================================================
# Final Save
# ============================================================================

print("\n" + "=" * 70)
print("Saving final results...")
print("=" * 70)

results_hierarchical.to_csv(
    f'experiment_results/{out_dir}/results_hierarchical.csv',
    index=False
)

print("\nAll results saved successfully!")
print(f"Output directory: experiment_results/{out_dir}/")
print("\nFiles created:")
print("  - results_hierarchical.csv")

### Dayton

In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def simple_dst_fix(df: pd.DataFrame, start_at_midnight: bool = True) -> pd.DataFrame:
    df = df.copy()
    df["ds"] = pd.to_datetime(df["ds"])
    df = df.sort_values("ds")

    df = df[~df["ds"].duplicated(keep="first")]

    start = df["ds"].iloc[0]
    if start_at_midnight:
        start = start.normalize()
    end = df["ds"].iloc[-1]
    full_idx = pd.date_range(start, end, freq="h")

    out = df.set_index("ds").reindex(full_idx)

    num_cols = out.select_dtypes(include="number").columns
    out[num_cols] = out[num_cols].ffill()

    if out[num_cols].isna().any().any():
        out[num_cols] = out[num_cols].bfill()

    out = out.rename_axis("ds").reset_index()

    return out

In [3]:
df_dayton = pd.read_csv("data/DAYTON_hourly.csv")
df_dayton.rename(columns={'Datetime': 'ds', 'DAYTON_MW': 'y'}, inplace=True)
df_dayton['ds'] = pd.to_datetime(df_dayton['ds'], format='%Y-%m-%d %H:%M:%S')
df_dayton = simple_dst_fix(df_dayton)
out_dir = 'Dayton_results'

In [4]:
import prophet_linear_adjust_yearly_to_hourly

yearly_demand = df_dayton.groupby(df_dayton['ds'].dt.year)['y'].sum().reset_index()
date_start = pd.to_datetime('2005-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

results_prophet_yearly = []
i = 1

while True:
    print(i)
    result = prophet_linear_adjust_yearly_to_hourly.forecast_next_year_hourly(
        df_dayton, date_start, date_end, yearly_demand,
        bayesian_samples=0,
        manual=False,
        daily=True,
        weekly = True,
        yearly = True,
        monthly = True)

    results_prophet_yearly.append(result)

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    if i == 3:
        break
    else:
        i += 1

results_prophet_yearly = pd.concat(results_prophet_yearly, ignore_index=True)
results_prophet_yearly.to_csv('experiment_results/' + out_dir + '/results_prophet_yearly.csv', index=False)


1


06:05:00 - cmdstanpy - INFO - Chain [1] start processing
06:05:19 - cmdstanpy - INFO - Chain [1] done processing


2


06:10:58 - cmdstanpy - INFO - Chain [1] start processing
06:11:22 - cmdstanpy - INFO - Chain [1] done processing


3


06:17:42 - cmdstanpy - INFO - Chain [1] start processing
06:18:06 - cmdstanpy - INFO - Chain [1] done processing


In [5]:
import pandas as pd
import numpy as np
import torch
import prophet_yearly_to_daily
import RNN_fourier_RNN_uncertainty

train_config = RNN_fourier_RNN_uncertainty.training_config(n_epochs=30, device=torch.device("mps"))
fourier_conf = RNN_fourier_RNN_uncertainty.fourier_config(mode="matrix", K_weekly=3, K_monthly=6, K_yearly=10)

K_total = fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly
F_per_hour = 2 * K_total

if fourier_conf.mode == "vector":
    cont_dim = 1 + F_per_hour
    fourier_dim = F_per_hour
else:
    cont_dim = 1 + 24 * F_per_hour
    fourier_dim = F_per_hour

df_daily = df_dayton.copy()
df_daily['ds'] = pd.to_datetime(df_daily['ds'])
df_daily['date'] = df_daily['ds'].dt.date
df_daily_agg = df_daily.groupby('date')['y'].sum().reset_index()
df_daily_agg.columns = ['ds', 'y']
df_daily_agg['ds'] = pd.to_datetime(df_daily_agg['ds'])

yearly_demand = df_daily_agg.copy()
yearly_demand['year'] = yearly_demand['ds'].dt.year
yearly_demand = yearly_demand.groupby('year')['y'].sum().reset_index()
yearly_demand.columns = ['ds', 'y']

results_hybrid = pd.DataFrame({'ds': [], 'y_hat': [], 'y': []})

date_start = pd.to_datetime('2005-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

i = 1

while True:

    print(i)
    df_train_day = df_daily_agg.loc[(df_daily_agg['ds'] >= date_start) & (df_daily_agg['ds'] < date_end)].copy()
    df_test_day = df_daily_agg.loc[(df_daily_agg['ds'] >= date_end) & (df_daily_agg['ds'] < date_end + pd.DateOffset(years=1))].copy()

    # ===== STAGE 1: Prophet predicts daily loads for the next year =====
    result_prophet = prophet_yearly_to_daily.forecast_next_year_daily(
        df_train_day,
        df_test_day,
        yearly_demand,
        manual=False,
        weekly=True,
        monthly=True,
        yearly=True
    )
    result_prophet = result_prophet.sort_values('ds').reset_index()

    df_train_hour = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy().sort_values('ds').reset_index(drop=True)
    df_test_hour = df_dayton.loc[(df_dayton['ds'] >= date_end) & (df_dayton['ds'] < date_end + pd.DateOffset(years=1))].copy().sort_values('ds').reset_index(drop=True)

    model = RNN_fourier_RNN_uncertainty.RNN_fourier(
        cont_dim=cont_dim,
        fourier_dim=fourier_dim,
        xf_mode="matrix",
        d_model = 128,
        latent_dim=32,
        nhead=4
    )

    trainer = RNN_fourier_RNN_uncertainty.RNN_train_fourier(model, train_config, fourier_conf)
    trainer(df_train_hour)

    daily_prediction = result_prophet[['ds','adjusted_y']]
    daily_prediction.columns = ['ds', 'y']

    fake_hourly_load = []
    for _, row in daily_prediction.iterrows():
        day = row['ds']
        daily_val = row['y'] / 24
        for h in range(24):
            fake_hourly_load.append({
                'ds': pd.Timestamp(day) + pd.Timedelta(hours=h),
                'y': daily_val
            })

    # convert to DataFrame
    fake_hourly_load = pd.DataFrame(fake_hourly_load)

    forcast, _ = trainer.forecate(fake_hourly_load, deterministic = True)

    result = pd.DataFrame({'ds': df_test_hour['ds'], 'y_hat': forcast, 'y': df_test_hour['y']})

    results_hybrid = pd.concat([results_hybrid, result]).reset_index(drop=True)

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)
    if i  == 3:
        break

    else:
        i = i + 1

results_hybrid.to_csv('experiment_results/' + out_dir + '/results_prophet_rnn_hybrid.csv', index=False)

06:26:08 - cmdstanpy - INFO - Chain [1] start processing


1


Python(55170) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
06:26:09 - cmdstanpy - INFO - Chain [1] done processing
/Users/zero/Documents/GitHub/RNN_based_downscaling/RNN_fourier_RNN_uncertainty.py:495: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:837.)
  print(f"epoch {ep + 1} loss: {float(loss):.4f}")


epoch 1 loss: 0.1826
epoch 2 loss: 0.1148
epoch 3 loss: 0.0691
epoch 4 loss: 0.0510
epoch 5 loss: 0.0452
epoch 6 loss: 0.0447
epoch 7 loss: 0.0401
epoch 8 loss: 0.0392
epoch 9 loss: 0.0341
epoch 10 loss: 0.0426
epoch 11 loss: 0.0393
epoch 12 loss: 0.0362
epoch 13 loss: 0.0378
epoch 14 loss: 0.0339
epoch 15 loss: 0.0363
epoch 16 loss: 0.0385
epoch 17 loss: 0.0362
epoch 18 loss: 0.0372
epoch 19 loss: 0.0388
epoch 20 loss: 0.0366
epoch 21 loss: 0.0337
epoch 22 loss: 0.0337
epoch 23 loss: 0.0328
epoch 24 loss: 0.0318
epoch 25 loss: 0.0373
epoch 26 loss: 0.0335
epoch 27 loss: 0.0290
epoch 28 loss: 0.0298
epoch 29 loss: 0.0306
epoch 30 loss: 0.0357


/var/folders/3h/5yfj7yqx7lg98t310bfxxr3m0000gn/T/ipykernel_54370/2623238862.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_hybrid = pd.concat([results_hybrid, result]).reset_index(drop=True)
06:30:25 - cmdstanpy - INFO - Chain [1] start processing
Python(55290) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


...

epoch 1 loss: 0.2157
epoch 2 loss: 0.0810
epoch 3 loss: 0.0613
epoch 4 loss: 0.0473
epoch 5 loss: 0.0605
epoch 6 loss: 0.0651
epoch 7 loss: 0.0387
epoch 8 loss: 0.0430
epoch 9 loss: 0.0461
epoch 10 loss: 0.0465
epoch 11 loss: 0.0403
epoch 12 loss: 0.0479
epoch 13 loss: 0.0399
epoch 14 loss: 0.0370
epoch 15 loss: 0.0313
epoch 16 loss: 0.0489
epoch 17 loss: 0.0468
epoch 18 loss: 0.0365
epoch 19 loss: 0.0338
epoch 20 loss: 0.0329
epoch 21 loss: 0.0356
epoch 22 loss: 0.0354
epoch 23 loss: 0.0349
epoch 24 loss: 0.0341
epoch 25 loss: 0.0324
epoch 26 loss: 0.0380
epoch 27 loss: 0.0347
epoch 28 loss: 0.0344
epoch 29 loss: 0.0376
epoch 30 loss: 0.0302


06:34:40 - cmdstanpy - INFO - Chain [1] start processing


3


Python(55553) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
06:34:40 - cmdstanpy - INFO - Chain [1] done processing


epoch 1 loss: 0.1677
epoch 2 loss: 0.0778
epoch 3 loss: 0.0509
epoch 4 loss: 0.0485
epoch 5 loss: 0.0436
epoch 6 loss: 0.0449
epoch 7 loss: 0.0402
epoch 8 loss: 0.0379
epoch 9 loss: 0.0370
epoch 10 loss: 0.0433
epoch 11 loss: 0.0405
epoch 12 loss: 0.0510
epoch 13 loss: 0.0361
epoch 14 loss: 0.0346
epoch 15 loss: 0.0373
epoch 16 loss: 0.0377
epoch 17 loss: 0.0397
epoch 18 loss: 0.0360
epoch 19 loss: 0.0338
epoch 20 loss: 0.0381
epoch 21 loss: 0.0352
epoch 22 loss: 0.0357
epoch 23 loss: 0.0350
epoch 24 loss: 0.0349
epoch 25 loss: 0.0345
epoch 26 loss: 0.0329
epoch 27 loss: 0.0316
epoch 28 loss: 0.0316
epoch 29 loss: 0.0306
epoch 30 loss: 0.0356


In [ ]:
import pandas as pd
import numpy as np
import torch
import Hierarchical_RNN_2stage_uncertainty as hrnn
import os


# ============================================================================
# Configuration
# ============================================================================

fourier_config = hrnn.FourierConfig(
    K_yearly_to_daily=8,
    K_monthly_to_daily=6,
    K_weekly_to_daily=8,
    K_yearly_to_hourly=10,
    K_monthly_to_hourly=7,
    K_daily_to_hourly=8,
    K_weekly_to_hourly=3,
)

train_config = hrnn.TrainingConfig(
    base_epochs=200,
    yearly_to_daily_multiplier=6,
    daily_to_hourly_multiplier=1,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    lr=5e-4,
    lambda0=1e-6,
    lambdaf=1e-5,
    batch_size=32,
    T_seq_yearly=3,
    T_seq_daily=32,
    curriculum_start_prob=1.0,
    curriculum_end_prob=1.0
)

# ============================================================================
# Results Storage
# ============================================================================

results_hierarchical = pd.DataFrame({
    'year': [],
    'month': [],
    'day': [],
    'hour': [],
    'y_hat': [],
    'y': []
})

# ============================================================================
# Rolling Window Setup
# ============================================================================

date_start = pd.to_datetime('2005-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

print("=" * 70)
print("2-Stage Hierarchical RNN - Deterministic Prediction")
print("=" * 70)
print(f"Initial training window: {date_start.date()} to {date_end.date()}")
print(f"Window size: ~10 years")
print(f"Prediction: 1 year ahead (deterministic)")
print(f"T_seq_yearly: {train_config.T_seq_yearly}")
print(f"T_seq_daily: {train_config.T_seq_daily}")
print(f"Device: {train_config.device}")
print("=" * 70)

# Create output directory
os.makedirs(f'experiment_results/{out_dir}', exist_ok=True)

# ============================================================================
# Rolling Window Testing Loop
# ============================================================================

i = 1

while True:
    print("\n" + "=" * 70)
    print(f"Iteration {i}")
    print("=" * 70)

    # Get training data
    df_train = df_dayton.loc[(df_dayton['ds'] >= date_start) & (df_dayton['ds'] < date_end)].copy().sort_values(by='ds').reset_index(drop=True)

    # Get test year
    test_year_start = date_end
    test_year_end = date_end + pd.DateOffset(years=1)
    df_test = df_dayton.loc[(df_dayton['ds'] >= test_year_start) & (df_dayton['ds'] < test_year_end)].copy().sort_values(by='ds').reset_index(drop=True)

    if len(df_test) == 0:
        print("No more test data available. Stopping.")
        break

    print(f"Training: {date_start.date()} to {date_end.date()}")
    print(f"Testing:  {test_year_start.date()} to {test_year_end.date()}")
    print(f"Training samples: {len(df_train)}")
    print(f"Test samples: {len(df_test)}")

    # ========================================================================
    # Build and Train Model
    # ========================================================================

    print("\nBuilding model...")
    model_config = dict(
        fourier_config=fourier_config,
        training_config=train_config,
        latent_dim=36,
        d_model=128,
        nhead=4
    )
    downscaler = hrnn.build_model_dp(hrnn.HierarchicalDownscaler_2Stage, **model_config)

    # Train model using Approach 2 (Curriculum Learning)
    print("\nTraining model...")
    trainer = hrnn.Approach2_CurriculumLearning_2Stage(downscaler)
    trainer.train(df_train)

    # ========================================================================
    # Generate Deterministic Prediction
    # ========================================================================

    test_year_int = test_year_start.year
    yearly_sum = df_test['y'].sum()

    print(f"\nTest year: {test_year_int}")
    print(f"Actual yearly sum: {yearly_sum:.2f}")

    # Prepare historical data from training
    data_dict_train = downscaler.module.prepare_data(df_train) if hasattr(downscaler, 'module') else downscaler.prepare_data(df_train)

    # Get yearly history for Stage 1
    df_yearly_train = data_dict_train['yearly'].sort_values('year')
    historical_years = [{'year': row['year'], 'yearly_sum': row['yearly_sum']} for _, row in df_yearly_train.iterrows()]

    print(f"Using {len(historical_years)} historical years as context")

    print(f"\nGenerating deterministic prediction for year {test_year_int}...")
    predictor = hrnn.HierarchicalPredictor_2Stage(downscaler)

    y_hat = predictor.predict_yearly_to_hourly(
        yearly_sum=yearly_sum,
        year=test_year_int,
        historical_years=historical_years
    )

    print(f"Generated predictions of shape {y_hat.shape}")

    # ========================================================================
    # Align with Test Data
    # ========================================================================

    n_hours_actual = len(df_test)
    n_hours_pred = len(y_hat)

    if n_hours_pred > n_hours_actual:
        y_hat = y_hat[:n_hours_actual]
        print(f"Trimmed predictions from {n_hours_pred} to {n_hours_actual} hours")
    elif n_hours_pred < n_hours_actual:
        # Pad if needed
        padding_value = y_hat.mean()
        pad_width = n_hours_actual - n_hours_pred
        y_hat = np.pad(y_hat, (0, pad_width), 'constant', constant_values=padding_value)
        print(f"Padded predictions from {n_hours_pred} to {n_hours_actual} hours")

    # ========================================================================
    # Store Results
    # ========================================================================

    y_true = df_test['y'].values

    result = pd.DataFrame({
        'year': df_test['ds'].dt.year,
        'month': df_test['ds'].dt.month,
        'day': df_test['ds'].dt.day,
        'hour': df_test['ds'].dt.hour,
        'y_hat': y_hat,
        'y': y_true
    })

    # Append to results
    results_hierarchical = pd.concat(
        [results_hierarchical, result],
        ignore_index=True
    )

    # ========================================================================
    # Save Intermediate Results
    # ========================================================================

    if i % 1 == 0:
        print(f"\nSaving intermediate results...")
        results_hierarchical.to_csv(
            f'experiment_results/{out_dir}/results_hierarchical.csv',
            index=False
        )

    # ========================================================================
    # Move Window Forward
    # ========================================================================

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    # Stop after 3 years of testing
    if i == 3:
        print('\n' + '=' * 70)
        print('Finished 3 iterations!')
        print('=' * 70)
        break
    else:
        i += 1

# ============================================================================
# Final Save
# ============================================================================

print("\n" + "=" * 70)
print("Saving final results...")
print("=" * 70)

results_hierarchical.to_csv(
    f'experiment_results/{out_dir}/results_hierarchical.csv',
    index=False
)

print("\nAll results saved successfully!")
print(f"Output directory: experiment_results/{out_dir}/")
print("\nFiles created:")
print("  - results_hierarchical.csv")

### Deok

In [1]:
import pandas as pd
import numpy as np
from prophet import Prophet
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def simple_dst_fix(df: pd.DataFrame, start_at_midnight: bool = True) -> pd.DataFrame:
    df = df.copy()
    df["ds"] = pd.to_datetime(df["ds"])
    df = df.sort_values("ds")

    df = df[~df["ds"].duplicated(keep="first")]

    start = df["ds"].iloc[0]
    if start_at_midnight:
        start = start.normalize()
    end = df["ds"].iloc[-1]
    full_idx = pd.date_range(start, end, freq="h")

    out = df.set_index("ds").reindex(full_idx)

    num_cols = out.select_dtypes(include="number").columns
    out[num_cols] = out[num_cols].ffill()

    if out[num_cols].isna().any().any():
        out[num_cols] = out[num_cols].bfill()

    out = out.rename_axis("ds").reset_index()

    return out

In [3]:
df_deok = pd.read_csv("data/DEOK_hourly.csv")
df_deok.rename(columns={'Datetime': 'ds', 'DEOK_MW': 'y'}, inplace=True)
df_deok['ds'] = pd.to_datetime(df_deok['ds'], format='%Y-%m-%d %H:%M:%S')
df_deok = simple_dst_fix(df_deok)
out_dir = 'Deok_results'

In [4]:
import prophet_linear_adjust_yearly_to_hourly

yearly_demand = df_deok.groupby(df_deok['ds'].dt.year)['y'].sum().reset_index()
date_start = pd.to_datetime('2012-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

results_prophet_yearly = []
i = 1

while True:
    print(i)
    result = prophet_linear_adjust_yearly_to_hourly.forecast_next_year_hourly(
        df_deok, date_start, date_end, yearly_demand,
        bayesian_samples=0,
        manual=False,
        daily=True,
        weekly = True,
        yearly = True,
        monthly = True)

    results_prophet_yearly.append(result)

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    if i == 3:
        break
    else:
        i += 1

results_prophet_yearly = pd.concat(results_prophet_yearly, ignore_index=True)
results_prophet_yearly.to_csv('experiment_results/' + out_dir + '/results_prophet_yearly.csv', index=False)


1


07:12:08 - cmdstanpy - INFO - Chain [1] start processing
07:12:14 - cmdstanpy - INFO - Chain [1] done processing


2


07:17:34 - cmdstanpy - INFO - Chain [1] start processing
07:17:40 - cmdstanpy - INFO - Chain [1] done processing


3


07:23:51 - cmdstanpy - INFO - Chain [1] start processing
07:23:57 - cmdstanpy - INFO - Chain [1] done processing


In [6]:
import pandas as pd
import numpy as np
import torch
import prophet_yearly_to_daily
import RNN_fourier_RNN_uncertainty

train_config = RNN_fourier_RNN_uncertainty.training_config(n_epochs=30, device=torch.device("mps"))
fourier_conf = RNN_fourier_RNN_uncertainty.fourier_config(mode="matrix", K_weekly=3, K_monthly=6, K_yearly=10)

K_total = fourier_conf.K_weekly + fourier_conf.K_monthly + fourier_conf.K_yearly
F_per_hour = 2 * K_total

if fourier_conf.mode == "vector":
    cont_dim = 1 + F_per_hour
    fourier_dim = F_per_hour
else:
    cont_dim = 1 + 24 * F_per_hour
    fourier_dim = F_per_hour

df_daily = df_deok.copy()
df_daily['ds'] = pd.to_datetime(df_daily['ds'])
df_daily['date'] = df_daily['ds'].dt.date
df_daily_agg = df_daily.groupby('date')['y'].sum().reset_index()
df_daily_agg.columns = ['ds', 'y']
df_daily_agg['ds'] = pd.to_datetime(df_daily_agg['ds'])

yearly_demand = df_daily_agg.copy()
yearly_demand['year'] = yearly_demand['ds'].dt.year
yearly_demand = yearly_demand.groupby('year')['y'].sum().reset_index()
yearly_demand.columns = ['ds', 'y']

results_hybrid = pd.DataFrame({'ds': [], 'y_hat': [], 'y': []})

date_start = pd.to_datetime('2012-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

i = 1

while True:

    print(i)
    df_train_day = df_daily_agg.loc[(df_daily_agg['ds'] >= date_start) & (df_daily_agg['ds'] < date_end)].copy()
    df_test_day = df_daily_agg.loc[(df_daily_agg['ds'] >= date_end) & (df_daily_agg['ds'] < date_end + pd.DateOffset(years=1))].copy()

    # ===== STAGE 1: Prophet predicts daily loads for the next year =====
    result_prophet = prophet_yearly_to_daily.forecast_next_year_daily(
        df_train_day,
        df_test_day,
        yearly_demand,
        manual=False,
        weekly=True,
        monthly=True,
        yearly=True
    )
    result_prophet = result_prophet.sort_values('ds').reset_index()

    df_train_hour = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy().sort_values('ds').reset_index(drop=True)
    df_test_hour = df_deok.loc[(df_deok['ds'] >= date_end) & (df_deok['ds'] < date_end + pd.DateOffset(years=1))].copy().sort_values('ds').reset_index(drop=True)

    model = RNN_fourier_RNN_uncertainty.RNN_fourier(
        cont_dim=cont_dim,
        fourier_dim=fourier_dim,
        xf_mode="matrix",
        d_model = 128,
        latent_dim=32,
        nhead=4
    )

    trainer = RNN_fourier_RNN_uncertainty.RNN_train_fourier(model, train_config, fourier_conf)
    trainer(df_train_hour)

    daily_prediction = result_prophet[['ds','adjusted_y']]
    daily_prediction.columns = ['ds', 'y']

    fake_hourly_load = []
    for _, row in daily_prediction.iterrows():
        day = row['ds']
        daily_val = row['y'] / 24
        for h in range(24):
            fake_hourly_load.append({
                'ds': pd.Timestamp(day) + pd.Timedelta(hours=h),
                'y': daily_val
            })

    # convert to DataFrame
    fake_hourly_load = pd.DataFrame(fake_hourly_load)

    forcast, _ = trainer.forecate(fake_hourly_load, deterministic = True)

    result = pd.DataFrame({'ds': df_test_hour['ds'], 'y_hat': forcast, 'y': df_test_hour['y']})

    results_hybrid = pd.concat([results_hybrid, result]).reset_index(drop=True)

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    if i  == 3:
        break

    else:
        i = i + 1

results_hybrid.to_csv('experiment_results/' + out_dir + '/results_prophet_rnn_hybrid.csv', index=False)

07:30:59 - cmdstanpy - INFO - Chain [1] start processing
Python(57461) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
07:31:00 - cmdstanpy - INFO - Chain [1] done processing


1


/Users/zero/Documents/GitHub/RNN_based_downscaling/RNN_fourier_RNN_uncertainty.py:495: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:837.)
  print(f"epoch {ep + 1} loss: {float(loss):.4f}")


epoch 1 loss: 0.4490
epoch 2 loss: 0.3370
epoch 3 loss: 0.2543
epoch 4 loss: 0.2151
epoch 5 loss: 0.2006
epoch 6 loss: 0.2084
epoch 7 loss: 0.1880
epoch 8 loss: 0.1303
epoch 9 loss: 0.1319
epoch 10 loss: 0.1414
epoch 11 loss: 0.1184
epoch 12 loss: 0.0698
epoch 13 loss: 0.0660
epoch 14 loss: 0.0632
epoch 15 loss: 0.0692
epoch 16 loss: 0.0644
epoch 17 loss: 0.0765
epoch 18 loss: 0.0566
epoch 19 loss: 0.0606
epoch 20 loss: 0.0558
epoch 21 loss: 0.0605
epoch 22 loss: 0.0605
epoch 23 loss: 0.0541
epoch 24 loss: 0.0569
epoch 25 loss: 0.0582
epoch 26 loss: 0.0500
epoch 27 loss: 0.0534
epoch 28 loss: 0.0605
epoch 29 loss: 0.0584
epoch 30 loss: 0.0454


/var/folders/3h/5yfj7yqx7lg98t310bfxxr3m0000gn/T/ipykernel_56714/2669931423.py:92: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_hybrid = pd.concat([results_hybrid, result]).reset_index(drop=True)
07:32:17 - cmdstanpy - INFO - Chain [1] start processing
Python(57518) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


2


07:32:17 - cmdstanpy - INFO - Chain [1] done processing


epoch 1 loss: 0.6786
epoch 2 loss: 0.3461
epoch 3 loss: 0.2497
epoch 4 loss: 0.2290
epoch 5 loss: 0.1850
epoch 6 loss: 0.1247
epoch 7 loss: 0.1934
epoch 8 loss: 0.1004
epoch 9 loss: 0.0777
epoch 10 loss: 0.0702
epoch 11 loss: 0.0746
epoch 12 loss: 0.0632
epoch 13 loss: 0.0650
epoch 14 loss: 0.0703
epoch 15 loss: 0.0635
epoch 16 loss: 0.0510
epoch 17 loss: 0.0534
epoch 18 loss: 0.0537
epoch 19 loss: 0.0492
epoch 20 loss: 0.0575
epoch 21 loss: 0.0541
epoch 22 loss: 0.0466
epoch 23 loss: 0.0516
epoch 24 loss: 0.0454
epoch 25 loss: 0.0515
epoch 26 loss: 0.0583
epoch 27 loss: 0.0497
epoch 28 loss: 0.0474
epoch 29 loss: 0.0462
epoch 30 loss: 0.0453


07:33:40 - cmdstanpy - INFO - Chain [1] start processing
Python(57599) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
07:33:40 - cmdstanpy - INFO - Chain [1] done processing


3
epoch 1 loss: 0.5050
epoch 2 loss: 0.2192
epoch 3 loss: 0.1911
epoch 4 loss: 0.1137
epoch 5 loss: 0.0955
epoch 6 loss: 0.0971
epoch 7 loss: 0.0749
epoch 8 loss: 0.0720
epoch 9 loss: 0.0630
epoch 10 loss: 0.0712
epoch 11 loss: 0.0627
epoch 12 loss: 0.0545
epoch 13 loss: 0.0619
epoch 14 loss: 0.0865
epoch 15 loss: 0.0469
epoch 16 loss: 0.0427
epoch 17 loss: 0.0537
epoch 18 loss: 0.0670
epoch 19 loss: 0.0601
epoch 20 loss: 0.0503
epoch 21 loss: 0.0678
epoch 22 loss: 0.0424
epoch 23 loss: 0.0453
epoch 24 loss: 0.0416
epoch 25 loss: 0.0475
epoch 26 loss: 0.0408
epoch 27 loss: 0.0433
epoch 28 loss: 0.0383
epoch 29 loss: 0.0375
epoch 30 loss: 0.0313


In [ ]:
import pandas as pd
import numpy as np
import torch
import Hierarchical_RNN_2stage_uncertainty as hrnn
import os


# ============================================================================
# Configuration
# ============================================================================

fourier_config = hrnn.FourierConfig(
    K_yearly_to_daily=8,
    K_monthly_to_daily=6,
    K_weekly_to_daily=8,
    K_yearly_to_hourly=10,
    K_monthly_to_hourly=7,
    K_daily_to_hourly=8,
    K_weekly_to_hourly=3,
)

train_config = hrnn.TrainingConfig(
    base_epochs=200,
    yearly_to_daily_multiplier=6,
    daily_to_hourly_multiplier=1,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    lr=5e-4,
    lambda0=1e-6,
    lambdaf=1e-5,
    batch_size=32,
    T_seq_yearly=3,
    T_seq_daily=32,
    curriculum_start_prob=1.0,
    curriculum_end_prob=1.0
)

# ============================================================================
# Results Storage
# ============================================================================

results_hierarchical = pd.DataFrame({
    'year': [],
    'month': [],
    'day': [],
    'hour': [],
    'y_hat': [],
    'y': []
})

# ============================================================================
# Rolling Window Setup
# ============================================================================

date_start = pd.to_datetime('2012-01-01 00:00:00')
date_end = pd.to_datetime('2015-01-01 00:00:00')

print("=" * 70)
print("2-Stage Hierarchical RNN - Deterministic Prediction")
print("=" * 70)
print(f"Initial training window: {date_start.date()} to {date_end.date()}")
print(f"Window size: ~10 years")
print(f"Prediction: 1 year ahead (deterministic)")
print(f"T_seq_yearly: {train_config.T_seq_yearly}")
print(f"T_seq_daily: {train_config.T_seq_daily}")
print(f"Device: {train_config.device}")
print("=" * 70)

# Create output directory
os.makedirs(f'experiment_results/{out_dir}', exist_ok=True)

# ============================================================================
# Rolling Window Testing Loop
# ============================================================================

i = 1

while True:
    print("\n" + "=" * 70)
    print(f"Iteration {i}")
    print("=" * 70)

    # Get training data
    df_train = df_deok.loc[(df_deok['ds'] >= date_start) & (df_deok['ds'] < date_end)].copy().sort_values(by='ds').reset_index(drop=True)

    # Get test year
    test_year_start = date_end
    test_year_end = date_end + pd.DateOffset(years=1)
    df_test = df_deok.loc[(df_deok['ds'] >= test_year_start) & (df_deok['ds'] < test_year_end)].copy().sort_values(by='ds').reset_index(drop=True)

    if len(df_test) == 0:
        print("No more test data available. Stopping.")
        break

    print(f"Training: {date_start.date()} to {date_end.date()}")
    print(f"Testing:  {test_year_start.date()} to {test_year_end.date()}")
    print(f"Training samples: {len(df_train)}")
    print(f"Test samples: {len(df_test)}")

    # ========================================================================
    # Build and Train Model
    # ========================================================================

    print("\nBuilding model...")
    model_config = dict(
        fourier_config=fourier_config,
        training_config=train_config,
        latent_dim=36,
        d_model=128,
        nhead=4
    )
    downscaler = hrnn.build_model_dp(hrnn.HierarchicalDownscaler_2Stage, **model_config)

    # Train model using Approach 2 (Curriculum Learning)
    print("\nTraining model...")
    trainer = hrnn.Approach2_CurriculumLearning_2Stage(downscaler)
    trainer.train(df_train)

    # ========================================================================
    # Generate Deterministic Prediction
    # ========================================================================

    test_year_int = test_year_start.year
    yearly_sum = df_test['y'].sum()

    print(f"\nTest year: {test_year_int}")
    print(f"Actual yearly sum: {yearly_sum:.2f}")

    # Prepare historical data from training
    data_dict_train = downscaler.module.prepare_data(df_train) if hasattr(downscaler, 'module') else downscaler.prepare_data(df_train)

    # Get yearly history for Stage 1
    df_yearly_train = data_dict_train['yearly'].sort_values('year')
    historical_years = [{'year': row['year'], 'yearly_sum': row['yearly_sum']} for _, row in df_yearly_train.iterrows()]

    print(f"Using {len(historical_years)} historical years as context")

    print(f"\nGenerating deterministic prediction for year {test_year_int}...")
    predictor = hrnn.HierarchicalPredictor_2Stage(downscaler)

    y_hat = predictor.predict_yearly_to_hourly(
        yearly_sum=yearly_sum,
        year=test_year_int,
        historical_years=historical_years
    )

    print(f"Generated predictions of shape {y_hat.shape}")

    # ========================================================================
    # Align with Test Data
    # ========================================================================

    n_hours_actual = len(df_test)
    n_hours_pred = len(y_hat)

    if n_hours_pred > n_hours_actual:
        y_hat = y_hat[:n_hours_actual]
        print(f"Trimmed predictions from {n_hours_pred} to {n_hours_actual} hours")
    elif n_hours_pred < n_hours_actual:
        # Pad if needed
        padding_value = y_hat.mean()
        pad_width = n_hours_actual - n_hours_pred
        y_hat = np.pad(y_hat, (0, pad_width), 'constant', constant_values=padding_value)
        print(f"Padded predictions from {n_hours_pred} to {n_hours_actual} hours")

    # ========================================================================
    # Store Results
    # ========================================================================

    y_true = df_test['y'].values

    result = pd.DataFrame({
        'year': df_test['ds'].dt.year,
        'month': df_test['ds'].dt.month,
        'day': df_test['ds'].dt.day,
        'hour': df_test['ds'].dt.hour,
        'y_hat': y_hat,
        'y': y_true
    })

    # Append to results
    results_hierarchical = pd.concat(
        [results_hierarchical, result],
        ignore_index=True
    )

    # ========================================================================
    # Save Intermediate Results
    # ========================================================================

    if i % 1 == 0:
        print(f"\nSaving intermediate results...")
        results_hierarchical.to_csv(
            f'experiment_results/{out_dir}/results_hierarchical.csv',
            index=False
        )

    # ========================================================================
    # Move Window Forward
    # ========================================================================

    date_start += pd.DateOffset(years=1)
    date_end += pd.DateOffset(years=1)

    # Stop after 3 years of testing
    if i == 3:
        print('\n' + '=' * 70)
        print('Finished 3 iterations!')
        print('=' * 70)
        break
    else:
        i += 1

# ============================================================================
# Final Save
# ============================================================================

print("\n" + "=" * 70)
print("Saving final results...")
print("=" * 70)

results_hierarchical.to_csv(
    f'experiment_results/{out_dir}/results_hierarchical.csv',
    index=False
)

print("\nAll results saved successfully!")
print(f"Output directory: experiment_results/{out_dir}/")
print("\nFiles created:")
print("  - results_hierarchical.csv")